In [ ]:
# Import necessary libraries

import psycopg2
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd

In [ ]:
# Create a geo data frame from districts_enhanced.geojson (already clean data)

gdf = gpd.read_file("districts_enhanced.geojson")

In [16]:
# Create connection and engine to establish a connection

with open("../_db_login/layered_db_url.txt", "r") as file:
    DATABASE_URL = file.read().strip()

engine = create_engine(DATABASE_URL)

In [ ]:
# Set search path to avoid PostGIS issues during table creation/upload

with engine.connect() as conn:
    conn.execute(text('''SET search_path TO public, berlin_data;
'''))
    conn.commit()

In [ ]:
# Create empty districts table with constraints (PK)

with engine.connect() as conn:
    conn.execute(text('''
    CREATE TABLE berlin_data.districts (
    district_id VARCHAR(2) NOT NULL,
    district VARCHAR(32) NOT NULL,
    geometry geometry(MULTIPOLYGON, 4326) NOT NULL,
    CONSTRAINT district_id_PK PRIMARY KEY (district_id)
    );
    '''))
    conn.commit()

In [ ]:
# Use to_postgis (not to_sql) to preserve geometry type during upload

from geoalchemy2 import Geometry

gdf.to_postgis(
    name='districts',
    con=engine,
    schema='berlin_data',
    if_exists='append',  # 'append' if table exists and you want to add rows and keep table structure (constraints and so on...)
    index=False          # 'replace' replaces the table structure (all constraints will be gone!)
)